In [1]:

from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(
    os.environ.get('WORK_DIR')) #type: ignore
sys.path.append(
    os.environ.get('DBASE_DIR')) #type: ignore
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from data import HistoricTradeDataHandler
from event import *
from queue import Queue
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import BBandsTrend2
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from backtest import OptionSignalBacktest
from copy import deepcopy
%load_ext autoreload
%autoreload 2


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-02-16 21:10:59 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [3]:
trades_ = pd.read_csv('profitable_trades.csv').iloc[:, 1:]
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,27.0,504.0,568.0,336.262811,314.029999,-600.285939,-0.066117,2023-07-05,2023-10-04,91 days,MSFT
1,9.0,504.0,571.0,52.904521,51.693333,-10.900697,-0.022894,2023-07-05,2023-10-09,96 days,WMT
2,37.0,504.0,584.0,192.240502,170.369995,-809.208768,-0.113766,2023-07-05,2023-10-26,113 days,AAPL
3,21.0,504.0,584.0,130.695846,120.629997,-211.382813,-0.077017,2023-07-05,2023-10-26,113 days,AMZN
4,42.0,504.0,585.0,279.795877,210.600006,-2906.226592,-0.247308,2023-07-05,2023-10-27,114 days,TSLA
5,33.0,504.0,672.0,120.480208,132.740005,404.573332,0.101758,2023-07-05,2024-03-05,244 days,GOOG
6,73.0,504.0,753.0,116.506348,161.250000,3266.286562,0.384045,2023-07-05,2024-07-01,362 days,AMD
7,64.0,504.0,753.0,87.043588,160.820007,4721.690846,0.847580,2023-07-05,2024-07-01,362 days,AVGO
8,21.0,504.0,753.0,463.717356,911.770020,9409.105932,0.966219,2023-07-05,2024-07-01,362 days,LLY
9,422.0,504.0,753.0,42.282471,123.470001,34261.137831,1.920123,2023-07-05,2024-07-01,362 days,NVDA


In [7]:
import json
with open('profitable_weights.json', 'r') as f:
    weights = json.load(f)
ttrades__ = pd.read_csv('profitable_trades.csv').iloc[:, 1:]
ttrades__, weights

(     Size  EntryBar  ExitBar  EntryPrice   ExitPrice           PnL  ReturnPct  \
 0    27.0     504.0    568.0  336.262811  314.029999   -600.285939  -0.066117   
 1     9.0     504.0    571.0   52.904521   51.693333    -10.900697  -0.022894   
 2    37.0     504.0    584.0  192.240502  170.369995   -809.208768  -0.113766   
 3    21.0     504.0    584.0  130.695846  120.629997   -211.382813  -0.077017   
 4    42.0     504.0    585.0  279.795877  210.600006  -2906.226592  -0.247308   
 5    33.0     504.0    672.0  120.480208  132.740005    404.573332   0.101758   
 6    73.0     504.0    753.0  116.506348  161.250000   3266.286562   0.384045   
 7    64.0     504.0    753.0   87.043588  160.820007   4721.690846   0.847580   
 8    21.0     504.0    753.0  463.717356  911.770020   9409.105932   0.966219   
 9   422.0     504.0    753.0   42.282471  123.470001  34261.137831   1.920123   
 10    8.0     518.0    560.0  326.207752  306.160004   -160.381990  -0.061457   
 11   14.0     5

In [14]:
trades_ = ttrades__[:5]
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,27.0,504.0,568.0,336.262811,314.029999,-600.285939,-0.066117,2023-07-05,2023-10-04,91 days,MSFT
1,9.0,504.0,571.0,52.904521,51.693333,-10.900697,-0.022894,2023-07-05,2023-10-09,96 days,WMT
2,37.0,504.0,584.0,192.240502,170.369995,-809.208768,-0.113766,2023-07-05,2023-10-26,113 days,AAPL
3,21.0,504.0,584.0,130.695846,120.629997,-211.382813,-0.077017,2023-07-05,2023-10-26,113 days,AMZN
4,42.0,504.0,585.0,279.795877,210.600006,-2906.226592,-0.247308,2023-07-05,2023-10-27,114 days,TSLA


In [15]:
#Backtest class 
evb_backtest = OptionSignalBacktest(trades_)

In [16]:
evb_backtest.portfolio.weight_map = weights
evb_backtest.portfolio.weight_map

{'NVDA': 0.17403567829056413,
 'TSLA': 0.11550642768233775,
 'AMD': 0.0834286994973343,
 'LLY': 0.09757539849722477,
 'AAPL': 0.07043488643087037,
 'MSFT': 0.0908514723319269,
 'AVGO': 0.055093156449691004,
 'SBUX': 0.07221176940610122,
 'GOOG': 0.038799601494671494,
 'BAC': 0.04413788909693789,
 'AMZN': 0.027623397184346542,
 'MU': 0.018678439659805944,
 'QCOM': 0.018379279730883966,
 'DIS': 0.02993136919118428,
 'HD': 0.025585512620229425,
 'BA': 0.014360292506341181,
 'PFE': 0.013366729929548736,
 'INTC': 0.005000000000000001,
 'WMT': 0.005000000000000001}

In [17]:
signals = evb_backtest.bars.signal_df
signals

,Date,MSFT,WMT,AAPL,AMZN,TSLA
0,2023-07-05,1,1,1,1,1
1,2023-07-06,0,0,0,0,0
2,2023-07-07,0,0,0,0,0
3,2023-07-08,0,0,0,0,0
4,2023-07-09,0,0,0,0,0
...,...,...,...,...,...,...
110,2023-10-23,0,0,0,0,0
111,2023-10-24,0,0,0,0,0
112,2023-10-25,0,0,0,0,0
113,2023-10-26,0,0,-1,-1,0


In [18]:
signals_df = deepcopy(signals).set_index('Date')
signals_df[signals_df!=-1].sum().sum()

5.0

In [19]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

        Date  MSFT  WMT  AAPL  AMZN  TSLA
0 2023-07-05     1    1     1     1     1
Processing event: MARKET
Processing event: SIGNAL
                 long             short  close
100  MSFT20240621C355  MSFT20240621C365   4.85
Processing event: SIGNAL
Processing event: SIGNAL
                long             short  close
56  AAPL20240621C200  AAPL20240621C210  4.825
Processing event: SIGNAL
                long             short  close
45  AMZN20240621C130  AMZN20240621C140    4.9
Processing event: SIGNAL
                    long                short  close
182  TSLA20240621C293.33  TSLA20240621C306.67  4.975
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: FILL
Processing event: FILL
Processing event: FILL
Processing event: FILL
Event queue is empty, processed 14 event(s)
        Date  MSFT  WMT  AAPL  AMZN  TSLA
1 2023-07-06     0    0     0     0     0
Processing event: MARKET
Event queue is empty, processed 1 event(s

In [22]:
pd.set_option('display.max_rows', 100)

In [23]:
pd.DataFrame(evb_backtest.portfolio.all_holdings)

,MSFT,WMT,AAPL,AMZN,TSLA,datetime,cash,commission,total
0,0.000000,0.0,0.000000,0.000000,0.000000,2023-07-05,100000.000000,0.000000,100000.000000
1,9085.147233,0.0,7043.488643,2762.339718,11550.642768,2023-07-05,99695.450713,0.173496,130137.069076
2,8757.332642,0.0,7152.972923,2635.497589,11434.555906,2023-07-06,99695.450713,0.173496,129675.809773
3,8476.348707,0.0,6824.520084,2663.684728,11434.555906,2023-07-07,99695.450713,0.173496,129094.560140
4,7961.211493,0.0,6532.562006,2536.842599,10679.991303,2023-07-10,99695.450713,0.173496,127406.058114
5,7961.211493,0.0,6459.572486,2663.684728,11086.295320,2023-07-11,99695.450713,0.173496,127866.214741
6,8523.179363,0.0,6532.562006,2720.059008,11434.555906,2023-07-12,99695.450713,0.173496,128905.806997
7,8242.195428,0.0,6788.025324,2945.556128,11608.686199,2023-07-13,99695.450713,0.173496,129279.913793
8,7867.550181,0.0,6788.025324,2945.556128,12014.990216,2023-07-14,99695.450713,0.173496,129311.572564
9,8148.534116,0.0,7335.446722,2875.088278,12653.467957,2023-07-17,99695.450713,0.173496,130707.987787


In [14]:
stats.print_stats(30)
print(stream.getvalue())
with open('bactest_data.txt', 'w') as f:
  stream.seek(0)
  f.write(stream.read())
  f.flush()

         291176502 function calls (264139354 primitive calls) in 826.490 seconds

   Ordered by: cumulative time
   List reduced from 2981 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  826.504  413.252 /Users/chiemelienwanisobi/miniconda3/envs/openbb/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3541(run_code)
        2    0.000    0.000  826.504  413.252 {built-in method builtins.exec}
        1    0.000    0.000  826.504  826.504 /var/folders/j0/80hkbygd4lb27h9mw76gqzpw0000gn/T/ipykernel_43067/2161208997.py:1(<module>)
        1    0.051    0.051  826.504  826.504 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/backtest.py:37(run)
       62    0.001    0.000  664.107   10.711 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/portfolio.py:270(update_signal)
       62    0.012    0.000  664.102   10.711 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDr

In [23]:
evb_backtest.trades

,Ticker,PnL,EntryPrice,ExitPrice,ReturnPct,Quantity,EntryTime,ExitTime,Duration,Positions
0,MSFT,-4021.729957,485.231603,270.536235,-44.245957,18.732262,2023-07-05,2023-10-04,91,&L:MSFT20240621C355&S:MSFT20240621C365
1,AAPL,-5433.192644,482.278316,110.088399,-77.173264,14.597904,2023-07-05,2023-10-26,113,&L:AAPL20240621C200&S:AAPL20240621C210
2,AMZN,-570.587808,489.369516,388.155308,-20.682573,5.637428,2023-07-05,2023-10-26,113,&L:AMZN20240621C130&S:AMZN20240621C140
3,TSLA,-7212.238189,498.356369,187.716644,-62.332849,23.217372,2023-07-05,2023-10-27,114,&L:TSLA20240621C293.33&S:TSLA20240621C306.67
4,GOOG,570.885387,497.681533,570.882151,14.708325,7.798915,2023-07-05,2024-03-05,244,&L:GOOG20240621C127.5&S:GOOG20240621C137.5
5,AMD,17081.493376,489.487683,1492.731388,204.957906,17.026265,2023-07-05,2024-07-01,362,&L:AMD20240621C125&S:AMD20240621C140
6,BA,-399.175537,447.858690,322.933226,-27.893947,3.195310,2023-08-03,2023-09-11,39,&L:BA20240621C240&S:BA20240621C250
7,INTC,-217.222380,473.481778,266.664120,-43.680173,1.050309,2023-09-12,2023-10-24,42,&L:INTC20240621C37&S:INTC20240621C50
8,MU,-540.438854,447.419068,317.373543,-29.065709,4.155767,2023-09-14,2023-10-27,43,&L:MU20240621C70&S:MU20240621C80
9,AAPL,1575.913515,498.242601,609.895259,22.409296,14.114429,2023-11-03,2024-02-16,105,&L:AAPL20240920C180&S:AAPL20240920C190


In [33]:
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,27.0,504.0,568.0,336.262811,314.029999,-600.285939,-0.066117,2023-07-05,2023-10-04,91 days,MSFT
1,9.0,504.0,571.0,52.904521,51.693333,-10.900697,-0.022894,2023-07-05,2023-10-09,96 days,WMT
2,37.0,504.0,584.0,192.240502,170.369995,-809.208768,-0.113766,2023-07-05,2023-10-26,113 days,AAPL
3,21.0,504.0,584.0,130.695846,120.629997,-211.382813,-0.077017,2023-07-05,2023-10-26,113 days,AMZN
4,42.0,504.0,585.0,279.795877,210.600006,-2906.226592,-0.247308,2023-07-05,2023-10-27,114 days,TSLA
5,33.0,504.0,672.0,120.480208,132.740005,404.573332,0.101758,2023-07-05,2024-03-05,244 days,GOOG
6,73.0,504.0,753.0,116.506348,161.250000,3266.286562,0.384045,2023-07-05,2024-07-01,362 days,AMD
7,64.0,504.0,753.0,87.043588,160.820007,4721.690846,0.847580,2023-07-05,2024-07-01,362 days,AVGO
8,21.0,504.0,753.0,463.717356,911.770020,9409.105932,0.966219,2023-07-05,2024-07-01,362 days,LLY
9,422.0,504.0,753.0,42.282471,123.470001,34261.137831,1.920123,2023-07-05,2024-07-01,362 days,NVDA


In [32]:
from copy import deepcopy
test_data  =deepcopy(evb_backtest.portfolio.options_data['AAPL20240621C200'] - evb_backtest.portfolio.options_data['AAPL20240621C210'])
test_data[(test_data.index == '2023-07-05') | (test_data.index == '2023-10-26')]

,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,
2023-07-05,4.61,5.07,4.53,4.66,1207,-2,4.75,95,4.90,4.825,4.908133
2023-07-05,4.61,5.07,4.53,4.66,1207,-2,4.75,95,4.90,4.825,4.908133
2023-10-26,2.23,2.23,1.67,1.92,425,77,1.81,12,1.84,1.825,1.768220
2023-10-26,2.23,2.23,1.67,1.92,425,77,1.81,12,1.84,1.825,1.768220


In [33]:
evb_backtest.portfolio.all_positions


[{'GOOGL': {},
  'AMD': {},
  'MSFT': {},
  'datetime': Timestamp('2023-07-24 00:00:00')},
 {'GOOGL': {'position': {'long': ['GOOGL20240621P115'],
    'short': ['GOOGL20240621P100'],
    'trade_id': '&L:GOOGL20240621P115&S:GOOGL20240621P100',
    'close': 5.0},
   'quantity': 66.0,
   'market_value': 33000.0},
  'AMD': {'position': {'long': ['AMD20240621P110'],
    'short': ['AMD20240621P100'],
    'trade_id': '&L:AMD20240621P110&S:AMD20240621P100',
    'close': 4.75},
   'quantity': 69.47368421052632,
   'market_value': 33000.0},
  'MSFT': {},
  'datetime': Timestamp('2023-07-24 00:00:00')},
 {'GOOGL': {},
  'AMD': {},
  'MSFT': {},
  'datetime': Timestamp('2023-07-25 00:00:00')},
 {'GOOGL': {'position': {'long': ['GOOGL20240621C135'],
    'short': ['GOOGL20240621C147.5'],
    'trade_id': '&L:GOOGL20240621C135&S:GOOGL20240621C147.5',
    'close': 4.85},
   'quantity': 68.04121145934306,
   'market_value': 32999.98755778139},
  'AMD': {'position': {'long': ['AMD20240621P110'],
    'sho

In [34]:
evb_backtest.portfolio.options_data['GOOGL20240621P115'] - evb_backtest.portfolio.options_data['GOOGL20240621P100']

,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,
2023-07-24,4.16,4.13,4.05,4.07,-9,-134,5.85,38,4.15,5.000,5.381509
2023-07-24,4.16,4.13,4.05,4.07,-9,-134,5.85,38,4.15,5.000,5.381509
2023-07-25,4.25,4.25,3.93,3.91,-3198,2,3.30,27,4.35,3.825,3.857051
2023-07-25,4.25,4.25,3.93,3.91,-3198,2,3.30,27,4.35,3.825,3.857051
2023-07-26,2.88,3.12,2.88,3.12,-910,18,3.02,-191,3.05,3.035,2.938961
2023-07-26,2.88,3.12,2.88,3.12,-910,18,3.02,-191,3.05,3.035,2.938961
2023-07-27,2.88,2.85,2.70,2.85,-25,-23,3.07,14,3.10,3.085,3.114296
2023-07-27,2.88,2.85,2.70,2.85,-25,-23,3.07,14,3.10,3.085,3.114296
2023-07-28,2.66,2.69,2.65,2.64,174,62,1.10,6,2.77,1.935,1.839688


In [24]:
from trade.helpers.helper import generate_option_tick
import pandas as pd

tick = 'AAPL'
exp = '2024-03-12'
right = 'C'
strike = 145.0
option_tick = generate_option_tick(tick, right, exp, strike)
option_tick


'AAPL20240312000145C'

In [35]:
evb_backtest.portfolio.current_positions

{'GOOGL': {}, 'AMD': {}, 'MSFT': {}}

In [26]:
stats.print_stats(20)  # Show the top 20 functions by cumulative time
print(stream.getvalue())

         53676519 function calls (52386882 primitive calls) in 216.194 seconds

   Ordered by: cumulative time
   List reduced from 2937 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  216.195  108.097 /Users/chiemelienwanisobi/miniconda3/envs/openbb/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3541(run_code)
        2    0.000    0.000  216.195  108.097 {built-in method builtins.exec}
        1    0.000    0.000  216.195  216.195 /var/folders/j0/80hkbygd4lb27h9mw76gqzpw0000gn/T/ipykernel_40825/2620833521.py:1(<module>)
        1    0.003    0.003  216.195  216.195 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/backtest.py:55(run)
       21    0.000    0.000  215.775   10.275 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/portfolio.py:270(update_signal)
       21    0.003    0.000  215.775   10.275 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriv

In [27]:
trades = evb_backtest.portfolio.get_trades()
print(trades.to_string())

KeyError: 'option'

In [80]:
#Get all holdings
holdings = evb_backtest.get_all_holdings()
print(holdings.to_string())

               AMD    MSFT   AMZN   GOOGL   AAPL      cash  commission     total
datetime                                                                        
2024-03-05     0.0     0.0    0.0     0.0    0.0  100000.0         0.0  100000.0
2024-03-05     0.0     0.0    0.0     0.0    0.0  100000.0         0.0  100000.0
2024-03-05  3680.0     0.0    0.0     0.0    0.0   96320.0         0.0  100000.0
2024-03-05  3680.0  1624.0    0.0     0.0    0.0   94696.0         0.0  100000.0
2024-03-05  3680.0  1624.0  416.0     0.0    0.0   94280.0         0.0  100000.0
2024-03-06  4596.0  1402.0    0.0     0.0    0.0   94280.0         0.0  100278.0
2024-03-07  4804.0  1770.0    0.0     0.0    0.0   94280.0         0.0  100854.0
2024-03-08  4630.0  1880.0    0.0     0.0    0.0   94280.0         0.0  100790.0
2024-03-11  3746.0     0.0  380.0     0.0    0.0   94280.0         0.0   98406.0
2024-03-12  3310.0  1990.0  378.0     0.0    0.0   94280.0         0.0   99958.0
2024-03-13  3178.0  2096.0  

In [36]:
positions = evb_backtest.get_all_positions()
print(positions.to_string())

                             AMD                   AAPL                   MSFT                   GOOGL
datetime                                                                                              
2024-03-04                  None                   None                   None                    None
2024-03-04                  None                   None                   None                    None
2024-03-05  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-06  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-07  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-08  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-11  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-12  AMD-20240816-270.0-C                   None  MSFT-20240816-48

In [64]:
evb_backtest.portfolio.all_positions

[{'AMD': {'quantity': 0.0, 'option': None},
  'MSFT': {'quantity': 0.0, 'option': None},
  'AMZN': {'quantity': 0.0, 'option': None},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-04 00:00:00')},
 {'AMD': {'quantity': 0.0, 'option': None},
  'MSFT': {'quantity': 0.0, 'option': None},
  'AMZN': {'quantity': 0.0, 'option': None},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-04 00:00:00')},
 {'AMD': {'quantity': 2,
   'option': 'AMD-20240920-240.0-C',
   'entry_price': 2185.0},
  'MSFT': {'quantity': 2,
   'option': 'MSFT-20250117-540.0-C',
   'entry_price': 778.0},
  'AMZN': {'quantity': 2,
   'option': 'AMZN-20250321-205.0-C',
   'entry_price': 0.0},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-05 00:00:00')},
 {'AMD': {'quantity': 2,
   'option': 'AMD-20

In [7]:
from pprint import pprint

print('All positions:')
pprint(portfolio.all_positions)

print('\nCurrent holdings:')
pprint(portfolio.current_holdings)

print('\nCurrent positions:')
pprint(portfolio.current_positions)

All positions:
[{'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': '20240226'},
 {'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': Timestamp('2024-02-26 00:00:00')},
 {'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': Timestamp('2024-02-26 00:00:00')}]

Current holdings:
{'AAPL': 0.0,
 'AMD': 0.0,
 'AMZN': 0.0,
 'GOOGL': 0.0,
 'MSFT': 0.0,
 'cash': 100000,
 'commission': 0.0,
 'total': 100000}

Current positions:
{'AAPL': {'option': N

# Test RiskManager 

In [13]:
from EventDriven.riskmanager import RiskManager
from dbase.DataAPI.ThetaData import list_contracts, retrieve_option_ohlc, is_theta_data_retrieval_successful #type: ignore
import datetime
import pandas as pd
import pandas_market_calendars as mcal
import unittest
import numpy as np
import pprint as pp

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'NFLX']


#generate date range 
nyse = mcal.get_calendar('NYSE')
year_ago_date = datetime.datetime.now() - datetime.timedelta(days=365)
schedule = nyse.schedule(start_date=year_ago_date, end_date=datetime.datetime.now())
date_range = mcal.date_range(schedule, frequency='1D')
dates = [date.strftime('%Y-%m-%d') for date in date_range]




class RiskManagerOperations(unittest.TestCase):
    def __init__(self):
        self.events = Queue(maxsize=0)  
        self.bars = HistoricTradeDataHandler(self.events, trades)
        self.risk_manager = RiskManager(bars=self.bars, events=self.events, initial_capital=1000000)
        
    def test_order_picker(self):
        ticker = np.random.choice(tickers)
        contract_date = np.random.choice(dates)
        contracts = list_contracts(ticker, pd.to_datetime(contract_date).strftime('%Y%m%d'))
        self.assertTrue(is_theta_data_retrieval_successful(contracts))
        
        contract = contracts.sample()
        print(contract)
        contract_right = contract['right'].values[0]
        contract_expiration = f"{contract['expiration'].values[0]}"
        contract_strike = float(contract['strike'].values[0])
        max_close = np.random.randint(1, 10)
        
        #order settings 
        moneyness_width = np.random.uniform(0.05, 0.1)
        rel_strike_long = np.random.uniform(1.05, 1.3) 
        rel_strike_short = np.random.uniform(0.7, 0.95)
        dte = np.random.randint(30, 365)
        
        order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': rel_strike_long, 'dte': dte, 'moneyness_width': moneyness_width},
                {'direction': 'short', 'rel_strike': rel_strike_short, 'dte': dte, 'moneyness_width': moneyness_width} 
            ],
            'name': 'vertical_spread'
        }
       
        try:
            self.order = self.risk_manager.OrderPicker.get_order(ticker, contract_expiration, contract_right, max_close, order_settings)
            self.assertIsInstance(self.order, dict)
            self.assertIsInstance(self.order['long'], list)
            self.assertIsInstance(self.order['short'], list)
            self.assertGreater(len(self.order['long']), 0)
            self.assertGreater(len(self.order['short']), 0)
            self.assertIsInstance(self.order['close'], float)
        except AssertionError as e:
            print(f"AssertionError: {e}")
            pp.pprint(contract)
            print(f"Ticker: {ticker}")
            print(f"Contract Date: {contract_date}")
            print(f"Contract Right: {contract_right}")
            print(f"Contract Expiration: {contract_expiration}")
            print(f"Contract Strike: {contract_strike}")
            print(f"Max Close: {max_close}")
            print(f"Order Settings: {order_settings}")
            raise
        except Exception as e:
            print(f"Exception: {e}")
            pp.pprint(contract)
            print(f"Ticker: {ticker}")
            print(f"Contract Date: {contract_date}")
            print(f"Contract Right: {contract_right}")
            print(f"Contract Expiration: {contract_expiration}, type: {type(contract_expiration)}")
            print(f"Contract Strike: {contract_strike}, type: {type(contract_strike)}")
            print(f"Max Close: {max_close}")
            print(f"Order Settings: {order_settings}")
            raise


ops = RiskManagerOperations()
ops.test_order_picker()

      root  expiration  strike right
3444  NVDA    20240301   270.0     C
Exception: Cannot set a DataFrame with multiple columns to the single column option_id
      root  expiration  strike right
3444  NVDA    20240301   270.0     C
Ticker: NVDA
Contract Date: 2024-02-09
Contract Right: C
Contract Expiration: 20240301, type: <class 'str'>
Contract Strike: 270.0, type: <class 'float'>
Max Close: 2
Order Settings: {'type': 'spread', 'specifics': [{'direction': 'long', 'rel_strike': 1.2305088940545148, 'dte': 284, 'moneyness_width': 0.08149699520032436}, {'direction': 'short', 'rel_strike': 0.9364378888363725, 'dte': 284, 'moneyness_width': 0.08149699520032436}], 'name': 'vertical_spread'}


ValueError: Cannot set a DataFrame with multiple columns to the single column option_id

In [15]:
from trade.assets.Stock import Stock
NVDA = Stock('NVDA', run_chain = False)

In [14]:
def generate_option_to_buy(underlier: Stock, contract_time):
        """
        Buy an option based on the underlier.
        """
        time = contract_time
        next_day_time = time + pd.DateOffset(days=1)
        print(time, next_day_time)
        option_spot = underlier.spot(ts=True, ts_start = time, ts_end = next_day_time)
        option_spot = option_spot.iloc[0] 
        stock_price = option_spot['open']#use open price as spot price on the assumption of making trades at start of day
        oom_benchmark = 0.1#10% out of the money 
        expiry_benchmark = time + pd.DateOffset(months=5)
        oom_price = stock_price * (1 + oom_benchmark)
        time_str = time.strftime("%Y%m%d")
        contracts = list_contracts(underlier.ticker, time_str)
        print(contracts)
        contracts = contracts[contracts['right'] == 'C'] 
        
        
        #Filter out contracts that are out of the money
        contracts = contracts[contracts['strike'] >= oom_price]
        
        print('comparing expiry')
        print(type(expiry_benchmark))
        print(type(contracts['expiration']))
        #filter out contracts that are  below the expiry benchmark
        contracts = contracts[pd.to_datetime(contracts['expiration'], format="%Y%m%d") >= expiry_benchmark]
        
        #select a random contract to buy
        contract = contracts.sample(n=1); 
        
        return contract
                

In [8]:
appl = Stock('AAPL')
c_time = pd.to_datetime('2024-05-06', format="%Y-%m-%d")
c_time_next = c_time + pd.DateOffset(days=1)
aapl_spot = appl.spot(ts=True, ts_start = c_time, ts_end = c_time_next)
aapl_spot.iloc[0]['open']


182.35000610351562

In [15]:
option = generate_option_to_buy(appl, c_time)
option

2024-05-06 00:00:00 2024-05-07 00:00:00
      root  expiration  strike right
0     AAPL    20241018   220.0     C
1     AAPL    20250919   215.0     C
2     AAPL    20241115   220.0     C
3     AAPL    20250321   220.0     C
4     AAPL    20250321   220.0     P
...    ...         ...     ...   ...
1060  AAPL    20240621   220.0     C
1061  AAPL    20240621   220.0     P
1062  AAPL    20240719   220.0     C
1063  AAPL    20250620   215.0     C
1064  AAPL    20240816   220.0     C

[1065 rows x 4 columns]
comparing expiry
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas.core.series.Series'>


,root,expiration,strike,right
456,AAPL,20250117,235.0,C
